In [100]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import timedelta

# 1- Data Cleaning

In [101]:
df_loan = pd.read_csv("../Data/Raw/loan_applications.csv")
df_transaction = pd.read_csv("../Data/Raw/transactions.csv")

In [102]:
print("Loan Applications DataFrame Info:")
df_loan.info()
print("\n" + "-"*70 + "\n")
print("\nTransactions DataFrame Info:")
df_transaction.info()

Loan Applications DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   application_id             50000 non-null  object 
 1   customer_id                50000 non-null  object 
 2   application_date           50000 non-null  object 
 3   loan_type                  50000 non-null  object 
 4   loan_amount_requested      50000 non-null  float64
 5   loan_tenure_months         50000 non-null  int64  
 6   interest_rate_offered      50000 non-null  float64
 7   purpose_of_loan            50000 non-null  object 
 8   employment_status          50000 non-null  object 
 9   monthly_income             50000 non-null  float64
 10  cibil_score                50000 non-null  int64  
 11  existing_emis_monthly      50000 non-null  float64
 12  debt_to_income_ratio       50000 non-null  float64
 13  property_own

In [103]:
print("\nLoan Applications DataFrame Unique Value Counts:")
print(df_loan.nunique())
print("\n" + "-"*70 + "\n")
print("\nTransactions DataFrame Unique Value Counts:")
print(df_transaction.nunique())


Loan Applications DataFrame Unique Value Counts:
application_id               50000
customer_id                  18314
application_date              1096
loan_type                        5
loan_amount_requested         1312
loan_tenure_months               7
interest_rate_offered          983
purpose_of_loan                  7
employment_status                6
monthly_income                1101
cibil_score                    354
existing_emis_monthly          108
debt_to_income_ratio          3288
property_ownership_status        3
residential_address          18314
applicant_age                   45
gender                           3
number_of_dependents             5
loan_status                      4
fraud_flag                       2
fraud_type                       4
dtype: int64

----------------------------------------------------------------------


Transactions DataFrame Unique Value Counts:
transaction_id                       50000
customer_id                          1831

In [104]:
# Convert date columns to datetime
df_loan['application_date'] = pd.to_datetime(df_loan['application_date'])
df_transaction['transaction_date'] = pd.to_datetime(df_transaction['transaction_date'])

In [105]:
df_loan['fraud_type'].fillna('Not Fraudulent', inplace=True)

# value counts for fraud_type
print("\nValue counts for 'fraud_type' after handling missing values:")
print(df_loan['fraud_type'].value_counts())


Value counts for 'fraud_type' after handling missing values:
Not Fraudulent              48974
Synthetic Identity            276
Income Misrepresentation      270
Loan Stacking                 242
Transaction Laundering        238
Name: fraud_type, dtype: int64


# 2- Outiler treatment

In [106]:
# Identify numerical columns
numerical_cols = df_loan.select_dtypes(include=np.number).columns.tolist()
print(f"Numerical columns: {numerical_cols}")

Numerical columns: ['loan_amount_requested', 'loan_tenure_months', 'interest_rate_offered', 'monthly_income', 'cibil_score', 'existing_emis_monthly', 'debt_to_income_ratio', 'applicant_age', 'number_of_dependents', 'fraud_flag']


In [107]:
# Cap outliers using 1st and 99th percentiles
print("Capping outliers at 1st and 99th percentiles...")
for col in numerical_cols:
    lower_bound = df_loan[col].quantile(0.01)
    upper_bound = df_loan[col].quantile(0.99)
    df_loan[col] = df_loan[col].clip(lower=lower_bound, upper=upper_bound)

Capping outliers at 1st and 99th percentiles...


# 3- Feature Engineering

In [108]:
# 1. time-based features from application_date
df_loan['application_year'] = df_loan['application_date'].dt.year
df_loan['application_month'] = df_loan['application_date'].dt.month
df_loan['application_day_of_week'] = df_loan['application_date'].dt.dayofweek # Monday=0, Sunday=6

In [109]:
# 2. existing EMI to monthly income ratio  
epsilon = 1e-6
df_loan['existing_emi_to_income_ratio'] = (df_loan['existing_emis_monthly'] / (df_loan['monthly_income'] + epsilon)) * 100

# 3. loan amount to monthly income ratio
df_loan['loan_amount_to_income_ratio'] = (df_loan['loan_amount_requested'] / (df_loan['monthly_income'] + epsilon)) * 100

In [110]:
# 4. Feature: customer-level aggregates 
df_agg_transaction = df_transaction.groupby('customer_id').agg(
    transaction_count=('transaction_amount', 'count'),
    total_transaction_amount=('transaction_amount', 'sum'),
    avg_transaction_amount=('transaction_amount', 'mean'),
    max_transaction_amount=('transaction_amount', 'max'),
    fraud_count=('fraud_flag', 'sum'),
    fraud_rate=('fraud_flag', lambda x: x.mean() * 100),
    international_txn_rate=('is_international_transaction', lambda x: x.mean() * 100),
    unique_devices_used=('device_used', 'nunique')
).reset_index()

In [111]:
# Merge aggregated features into loan dataset
df_agg = pd.merge(df_loan, df_agg_transaction, on='customer_id', how='left')

In [112]:
%%time
# Features by costumers : aggregate transaction data before application date

# Merge transactions with loans on customer_id
df_txn_loan = pd.merge(
    df_transaction,
    df_loan[['application_id', 'customer_id', 'application_date']],
    on='customer_id',
    how='inner'
)

# Keep only transactions BEFORE application date
df_txn_loan = df_txn_loan[df_txn_loan['transaction_date'] < df_txn_loan['application_date']]

# Group by application_id and compute aggregates
df_txn_agg = df_txn_loan.groupby('application_id').agg(
    transaction_count_pre=('transaction_amount', 'count'),
    total_amount_pre=('transaction_amount', 'sum'),
    avg_amount_pre=('transaction_amount', 'mean'),
    fraud_count_pre=('fraud_flag', 'sum'),
    fraud_rate_pre=('fraud_flag', lambda x: x.mean() * 100)
).reset_index()

# Merge back to df_agg
df_agg = pd.merge(df_agg, df_txn_agg, on='application_id', how='left')


Wall time: 1.4 s


In [113]:
df_agg.columns

Index(['application_id', 'customer_id', 'application_date', 'loan_type',
       'loan_amount_requested', 'loan_tenure_months', 'interest_rate_offered',
       'purpose_of_loan', 'employment_status', 'monthly_income', 'cibil_score',
       'existing_emis_monthly', 'debt_to_income_ratio',
       'property_ownership_status', 'residential_address', 'applicant_age',
       'gender', 'number_of_dependents', 'loan_status', 'fraud_flag',
       'fraud_type', 'application_year', 'application_month',
       'application_day_of_week', 'existing_emi_to_income_ratio',
       'loan_amount_to_income_ratio', 'transaction_count',
       'total_transaction_amount', 'avg_transaction_amount',
       'max_transaction_amount', 'fraud_count', 'fraud_rate',
       'international_txn_rate', 'unique_devices_used',
       'transaction_count_pre', 'total_amount_pre', 'avg_amount_pre',
       'fraud_count_pre', 'fraud_rate_pre'],
      dtype='object')

In [114]:
ordered_columns = [
    # 1. Identifiers & Dates
    'application_id', 'customer_id', 'application_date',
    'application_year', 'application_month', 'application_day_of_week',

    # 2. Applicant Info
    'applicant_age', 'gender', 'number_of_dependents',
    'residential_address', 'property_ownership_status',
    'employment_status', 'monthly_income', 'cibil_score',

    # 3. Loan Details
    'loan_type', 'loan_amount_requested', 'loan_tenure_months',
    'interest_rate_offered', 'purpose_of_loan',
    'loan_amount_to_income_ratio', 'existing_emis_monthly',
    'existing_emi_to_income_ratio', 'debt_to_income_ratio',

    # 4. All-Time Transaction Aggregates
    'transaction_count', 'total_transaction_amount',
    'avg_transaction_amount', 'max_transaction_amount', 'fraud_count',
    'fraud_rate', 'international_txn_rate', 'unique_devices_used',

    # 5. Pre-Application Transaction Aggregates
    'transaction_count_pre', 'total_amount_pre',
    'avg_amount_pre','fraud_count_pre', 'fraud_rate_pre',

    # 6. Target Variables
    'loan_status', 'fraud_flag', 'fraud_type'
]

# Reorder the DataFrame
df_agg = df_agg[ordered_columns]

In [115]:
# For count & sum features
cols_to_fill_0 = [
    'transaction_count', 'total_transaction_amount', 'avg_transaction_amount',
    'max_transaction_amount','fraud_count', 'fraud_rate', 'international_txn_rate',
    'unique_devices_used','transaction_count_pre', 'total_amount_pre',
    'avg_amount_pre','fraud_count_pre', 'fraud_rate_pre'
]

df_agg[cols_to_fill_0] = df_agg[cols_to_fill_0].fillna(0)

In [116]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 39 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   application_id                50000 non-null  object        
 1   customer_id                   50000 non-null  object        
 2   application_date              50000 non-null  datetime64[ns]
 3   application_year              50000 non-null  int64         
 4   application_month             50000 non-null  int64         
 5   application_day_of_week       50000 non-null  int64         
 6   applicant_age                 50000 non-null  int64         
 7   gender                        50000 non-null  object        
 8   number_of_dependents          50000 non-null  int64         
 9   residential_address           50000 non-null  object        
 10  property_ownership_status     50000 non-null  object        
 11  employment_status           

In [117]:
# Save aggregated dataframe to CSV file

import os

# Define folder and file path
folder_path = '../Data/Processed'
file_name = 'loan_transactions_features.csv'
file_path = os.path.join(folder_path, file_name)

# Create folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Check if file exists and delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Old file '{file_name}' removed.")

# Save the DataFrame to CSV
df_agg.to_csv(file_path, index=False)
print(f"New file saved at: {file_path}")
print("✅ Feature engineering complete. Data saved to processed folder.")

Old file 'loan_transactions_features.csv' removed.
New file saved at: ../Data/Processed\loan_transactions_features.csv
✅ Feature engineering complete. Data saved to processed folder.
